In [1]:
import re

def parse_sc_fermi_output(output):
    data = {}
    regex = re.compile('SC Fermi level :\s+([-\d\.]+)')
    data['e_fermi'] = float(regex.findall(output)[0])
    regex = re.compile('(\w[()\w\s]+[\w\)])\s+:\s+([-+\d.E]+)\s+cm\^-3')
    data['defect species concentrations'] = {}
    for name, conc in regex.findall(output):
        data['defect species concentrations'][name] = float(conc)
    divider = '-'*56 +'\n'
    data['defect species'] = {}
    for defect_data in output.split(divider)[1:]:
        split_defect_data = defect_data.split(':')
        defect_species = split_defect_data[0].strip()
        data[defect_species] = {}
        for d in split_defect_data[2:]:
            values = [ float(s) for s in d.split() ]
            charge = values[0]
            data[defect_species][charge] = {}
            data[defect_species][charge]['concentration'] = values[1]
            data[defect_species][charge]['percentage'] = values[2]
    return data
    

In [2]:
import os
# set the location of the `sc-fermi` executable
sc_fermi_exec = '~/bin/sc-fermi'
output = os.popen(sc_fermi_exec).read()
print(output)
sc_fermi_output = parse_sc_fermi_output(output)

In [3]:
sc_fermi_output

{'e_fermi': -0.0322293767431,
 'defect species concentrations': {'n (electrons)': 1.493687426674e-24,
  'p (holes)': 1.898142087695e+19,
  'V_Ga': 1.898142087693e+19,
  'Ga_Sb': 1.24781971767e-85},
 'defect species': {},
 'V_Ga': {0.0: {'concentration': 1.024893014293e-101, 'percentage': 0.0},
  -1.0: {'concentration': 1.898142087693e+19, 'percentage': 100.0},
  -2.0: {'concentration': 5.142337688716e-100, 'percentage': 0.0},
  -3.0: {'concentration': 3.594428781987e-120, 'percentage': 0.0}},
 'Ga_Sb': {0.0: {'concentration': 1.236939790589e-92, 'percentage': 0.0},
  -1.0: {'concentration': 1.247819593689e-85, 'percentage': 100.0},
  -2.0: {'concentration': 2.874923404513e-95, 'percentage': 0.0}}}

In [4]:
# py-sc-fermi

from py_sc_fermi.defect_system import DefectSystem
from py_sc_fermi.defect_species import DefectSpecies
from py_sc_fermi.inputs import inputs_from_files
from py_sc_fermi.dos import DOS

import numpy as np

In [5]:
example_dir = './'

unitcell_filename = f'{example_dir}/unitcell.dat'
totdos_filename = f'{example_dir}/totdos.dat'
input_fermi_filename = f'{example_dir}/input-fermi.dat'

inputs = inputs_from_files(unitcell_filename=unitcell_filename, 
                           input_fermi_filename=input_fermi_filename, 
                           totdos_filename=totdos_filename)

Volume of cell: 57.78888409009906 A^3
Found non-spin polarised system...
Number of electrons in system: 18
Energy gap of system: 0.8084 eV
Temperature: 100.0 K
Number of defect species: 2
Reading non-spin-polarised DOS
Integration of DOS up to Fermi level: 13.669507297419017
Renormalised integrated DOS        : 18.0


In [6]:
defect_system = DefectSystem( defect_species=inputs['defect_species'],
                              volume=inputs['volume'],
                              dos=inputs['dos'], # DOS object
                              temperature=inputs['temperature'] )

print(defect_system)

defect_system.report()

DefectSystem
  nelect: 18 e
  egap:   0.8084 eV
  volume: 57.78888409009906 A^3
  temperature: 100.0 K

Contains defect species:

V_Ga, nsites=1
  q=+0.0, e=2.4451, deg=1
  q=-1.0, e=0.0265, deg=1
  q=-2.0, e=2.3469, deg=1
  q=-3.0, e=2.7146, deg=1

Ga_Sb, nsites=1
  q=+0.0, e=2.2649, deg=1
  q=-1.0, e=2.0937, deg=1
  q=-2.0, e=2.2527, deg=1

SC Fermi level :      -0.03222937674314075  (eV)

Concentrations:
n (electrons)  : 1.49368742667499e-24 cm^-3
p (holes)      : 1.8981420876945424e+19 cm^-3
V_Ga           : 1.8981420876946403e+19 cm^-3
Ga_Sb          : 1.2478197176710676e-85 cm^-3

Breakdown of concentrations for each defect charge state:
---------------------------------------------------------
V_Ga       : Charge Concentration(cm^-3) Total
           :  0.0  1.024893e-101          0.00 
           : -1.0  1.898142e+19          100.00 
           : -2.0  5.142338e-100          0.00 
           : -3.0  3.594429e-120          0.00 
--------------------------------------------------

/Users/bjm42/source/py-sc-fermi/py_sc_fermi/dos.py:84: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((e_fermi - edos)/kT))
/Users/bjm42/source/py-sc-fermi/py_sc_fermi/dos.py:87: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((edos - e_fermi)/kT))


In [7]:
import numpy as np
np.testing.assert_almost_equal(sc_fermi_output['e_fermi'], 
                               defect_system.get_sc_fermi(verbose=False),
                               decimal=13)